In [2]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 20240507

In [3]:
DATA_PATH = './mldata.csv'

In [4]:
df = pd.read_csv(DATA_PATH)

In [5]:
df.head()

,Unnamed: 0,stock_id,date,Advt_12M_Usd,Advt_3M_Usd,Advt_6M_Usd,Asset_Turnover,Bb_Yld,Bv,Capex_Ps_Cf,...,Total_Capital,Total_Debt,Total_Debt_Capital,Total_Liabilities_Total_Assets,Vol1Y_Usd,Vol3Y_Usd,R1M_Usd,R3M_Usd,R6M_Usd,R12M_Usd
0,0,13,2006-12-31,0.25,0.33,0.27,0.22,0.33,0.01,0.13,...,0.02,0.27,1.0,0.92,0.84,0.88,0.089,0.104,0.344,-0.041
1,1,13,2007-01-31,0.25,0.32,0.28,0.22,0.40,0.01,0.13,...,0.02,0.27,1.0,0.92,0.84,0.88,0.039,0.091,-0.011,-0.253
2,2,13,2007-02-28,0.26,0.30,0.30,0.22,0.15,0.01,0.13,...,0.02,0.27,1.0,0.92,0.87,0.88,-0.012,0.101,0.118,-0.366
3,3,17,2015-03-31,0.73,0.64,0.70,0.40,0.47,0.01,0.70,...,0.40,0.75,1.0,1.00,1.00,0.99,0.174,-0.144,-0.518,-0.376
4,4,17,2015-04-30,0.72,0.62,0.66,0.40,0.46,0.01,0.70,...,0.40,0.75,1.0,1.00,1.00,0.99,-0.106,-0.576,-0.535,-0.113


In [6]:
df = df[['stock_id', 'date', 'R1M_Usd']]
df.head()

,stock_id,date,R1M_Usd
0,13,2006-12-31,0.089
1,13,2007-01-31,0.039
2,13,2007-02-28,-0.012
3,17,2015-03-31,0.174
4,17,2015-04-30,-0.106


In [11]:
df = df.set_index(['stock_id', 'date']).unstack(level=0).sort_index().loc['2009-01-01':]

In [12]:
df.head()

R1M_Usd                                                          \
stock_id      1      2      3      4      5      6      7      8      9      
date                                                                         
2009-01-31  -0.103  0.360 -0.264 -0.061 -0.264 -0.197 -0.203 -0.367 -0.181   
2009-02-28   0.131  0.240  0.007  0.060  0.325  0.064  0.108  0.184  0.012   
2009-03-31   0.375  0.109  0.580 -0.040  0.309  0.087  0.203  0.655  0.218   
2009-04-30   0.216  0.078  0.081  0.038  0.076  0.043 -0.139 -0.010 -0.017   
2009-05-31  -0.093  0.027 -0.050  0.044 -0.116  0.013  0.082  0.072  0.029   

                   ...                                                   \
stock_id     10    ...   1203   1204   1205   1206   1207   1208   1209   
date               ...                                                    
2009-01-31 -0.193  ... -0.184 -0.050 -0.182 -0.239 -0.143  0.112 -0.057   
2009-02-28  0.070  ...  0.103  0.028  0.044  0.297 -0.004  0.078  0.297   
2009-03-31  0.010  ...  0.024  0.411  0.138  0.236  0.109 -0.005  0.142   
2009-04-30  0.026  ...  0.038 -0.152 -0.109 -0.135  0.073  0.066 -0.009   
2009-05-31  0.087  ... -0.025  0.086  0.050 -0.172  0.016  0.027 -0.057   

                                 
stock_id     1210   1211   1212  
date                             
2009-01-31 -0.258 -0.392 -0.375  
2009-02-28  0.085  0.088  0.040  
2009-03-31  0.236  0.369  0.446  
2009-04-30  0.026  0.092 -0.216  
2009-05-31  0.091 -0.061  0.366  

[5 rows x 1207 columns]

---
# AutoEncoder

In [13]:
N = df.shape[1]
N

1207

In [16]:
class AutoEncoder(nn.Module):
    def __init__(self, n, m):
        super().__init__()

        self.n = n
        self.m = m

        self.encoder = nn.Linear(in_features=self.n, out_features=self.m)
        self.decoder = nn.linear(in_features=self.m, out_features=self.n)

    def forward(self, X):
        x_1 = self.encoder(X)
        x_2 = self.decoder(x_1)

        return x_2